In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
# define the code 
code = """
from hypster import HP, config

@config
def nested_config(hp: HP):
    model = hp.select(['gpt-3', 'gpt-4'], default='gpt-3')

    generation = {
        'temperature': hp.number_input(0.7, min_value=0, max_value=1),
        'max_tokens': hp.number_input(50, min_value=1, max_value=100)
    }

    return {'model': model, 'generation': generation}

result = nested_config()
"""

In [10]:
import inspect
def func():
    print("5")

In [8]:
import inspect

def execute_with_source(code_string):
    # Create a dictionary to serve as the global namespace
    global_namespace = {}
    
    # Execute the code in the custom global namespace
    exec(code_string, global_namespace)
    
    # If 'func' is defined in the code, print its source
    if 'func' in global_namespace:
        func = global_namespace['func']
        print(inspect.getsource(func))

# Your original code as a string
code = """
import inspect

def func():
    print("5")

print(inspect.getsource(func))
"""

# Execute the code using our custom function
execute_with_source(code)

OSError: could not get source code

In [3]:
exec(code)

OSError: could not get source code

In [3]:
import hypster
from hypster import HP

@hypster.config
def my_config(hp: HP):
    chunking_strategy = hp.select(['paragraph', 'semantic', 'fixed'], default='paragraph')
    
    llm_model = hp.select({'haiku': 'claude-3-haiku-20240307', 
                           'sonnet': 'claude-3-5-sonnet-20240620',
                           'gpt-4o-mini': 'gpt-4o-mini'}, default='gpt-4o-mini')
    
    llm_config = {'temperature': hp.number_input(0), 
                  'max_tokens': hp.number_input(64)}
    
    system_prompt = hp.text_input('You are a helpful assistant. Answer with one word only')

In [4]:
results = my_config(final_vars=["chunking_strategy", "llm_config", "llm_model"], 
                    selections={"llm_model" : "haiku"}, 
                    overrides={"llm_config.temperature" : 0.5})

In [5]:
results

{'chunking_strategy': 'paragraph',
 'llm_config': {'temperature': 0, 'max_tokens': 64},
 'llm_model': 'claude-3-haiku-20240307'}

In [47]:
res, snp = my_config(selections={"a":"c"}, return_config_snapshot=True)#, overrides={"c" : 3, "d" : 4})

In [49]:
res

{'a': 'c', 'b': 1, 'c': 'ette', 'conf': {'a': 'c'}, 'd': 4}

In [48]:
snp

{'a': 'c', 'b': 1, 'c': 'ette', 'd': 4}

In [50]:
res == my_config(overrides=snp)#, overrides={"c" : 3, "d" : 4})

True

In [51]:
hypster.save(my_config, "my_config.py")

In [52]:
func = hypster.load("my_config.py")

In [53]:
import hypster
from hypster import HP

@hypster.config
def my_config_parent(hp: HP):
    import hypster
    my_config = hypster.load("my_config.py")
    my_conf = hp.propagate(my_config)
    a = hp.select(["a", "b", "c"], default="a")
    c = hp.select({"x": 1, "y": 2}, default="x")

In [54]:
result, snp = my_config_parent(final_vars=[], overrides={"my_conf.a": "44"}, return_config_snapshot=True)

In [55]:
result

{'my_config': <hypster.core.Hypster at 0x2450d756bf0>,
 'my_conf': {'a': '44', 'b': 1, 'c': 'ette', 'conf': {'a': '44'}, 'd': 4},
 'a': 'a',
 'c': 1}

In [56]:
snp

{'my_conf.a': '44',
 'my_conf.b': 1,
 'my_conf.c': 'ette',
 'my_conf.d': 4,
 'a': 'a',
 'c': 1}

In [57]:
my_config_parent(overrides=snp)

{'my_config': <hypster.core.Hypster at 0x2450d756cb0>,
 'my_conf': {'a': '44', 'b': 1, 'c': 'ette', 'conf': {'a': '44'}, 'd': 4},
 'a': 'a',
 'c': 1}

In [60]:
hypster.save(my_config, "my_config.py")

In [61]:
loaded_config = hypster.load("my_config.py")

In [62]:
result = loaded_config(final_vars=["a"], selections={"b": "y"}, overrides={"a": "c"})

In [63]:
loaded_config

In [64]:
result

{'a': 'c'}

In [ ]:
result = loaded_config(final_vars=["a"], selections={"b": "y"}, overrides={"a": "c"})
print(result)

In [2]:
from hypster import hypster

@hypster
def test(hp):
    a = hp.select(["a", "b", "c"], default="a")
    b = hp.select({"a" : "hello", "b" : "world"}, default="a")

In [3]:
res = test(final_vars = ["a", "b"], selections={"a" : "c"}, overrides = {"b" : "heyyy"})

In [4]:
res

{'a': 'a', 'b': 'hello'}

In [4]:
%%writefile configs.py
from hypster import Select, prep
from dataclasses import dataclass

class CacheInterface:
    pass

class DiskCache(CacheInterface):
    def __init__(self, path, cache_op: str):
        self.path = path
        self.cache_op = cache_op

class MemoryCache(CacheInterface):
    def __init__(self, max_size, cache_op: str):
        self.max_size = max_size
        self.cache_op = cache_op

class SqlCache(CacheInterface):
    def __init__(self, conn_str, table):
        self.conn_str = conn_str
        self.table = table

@dataclass
class CacheManager:
    cache: CacheInterface

cache_manager = prep(CacheManager(cache=Select("cache"))) #this can also be None


cache_op = "all"

max_size = 1000
cache__mem = prep(MemoryCache(max_size=max_size, cache_op=cache_op))

path = "data/cache"
cache__disk = prep(DiskCache(path=path, cache_op=cache_op))

cache__new = prep(SqlCache(conn_str="sqlite:///data/cache.db", 
                           table="cache"))

class OpenAiDriver:
    def __init__(self, model):
        self.model = model

class AnthropicDriver:
    def __init__(self, model):
        self.model = model

llm_driver = Select("llm_driver")
llm_driver__openai = prep(OpenAiDriver(model="gpt3.5"))
llm_driver__anthropic = prep(AnthropicDriver(model="claude3.5"))

Writing configs.py


In [10]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
%autoreload 2

In [17]:
from hypster import Builder
import configs

builder = Builder().with_modules(configs)
driver = builder.build()


In [18]:
# Visualize configuration tree
from hypster import visualize_config_tree
print("\nConfiguration tree:")
print(visualize_config_tree(driver.root))


Configuration tree:
└── root (root)
    ├── cache_manager (CacheManager)
    │   └── cache (Select): cache
    │       ├── mem (MemoryCache)
    │       │   ├── max_size (reference): 1000
    │       │   └── cache_op (reference): all
    │       ├── disk (DiskCache)
    │       │   ├── path (reference): data/cache
    │       │   └── cache_op (reference) [SHARED]
    │       └── new (SqlCache)
    │           ├── conn_str (value): sqlite:///data/cache.db
    │           └── table (value): cache
    └── llm_driver (Select): llm_driver
        ├── openai (OpenAiDriver)
        │   └── model (value): gpt3.5
        └── anthropic (AnthropicDriver)
            └── model (value) [SHARED]



In [34]:
final_vars = ["cache_manager", "llm_driver"]
selections = {"llm_driver": "anthropic", 
              "cache_manager.cache": "disk"}
overrides = {"llm_driver.anthropic.model": "claude3-opus"}
results = driver.instantiate(final_vars, selections, overrides)

In [35]:
results["cache_manager"].cache.path

'data/cache'

In [36]:
# Visualize configuration tree
from hypster import visualize_config_tree
print("\nConfiguration tree:")
print(visualize_config_tree(driver.root))


Configuration tree:
└── root (root)
    ├── cache_manager (CacheManager)
    │   └── cache (Select): cache
    │       └── disk (DiskCache)
    │           ├── path (reference): data/cache
    │           └── cache_op (reference): all
    ├── llm_driver (Select): llm_driver
    │   ├── openai (OpenAiDriver)
    │   │   └── model (value): gpt3.5
    │   └── anthropic (AnthropicDriver)
    │       └── model (value) [SHARED]
    ├── mem (MemoryCache)
    │   ├── max_size (reference): 1000
    │   └── cache_op (reference) [SHARED]
    ├── disk (DiskCache) [SHARED]
    └── new (SqlCache)
        ├── conn_str (value): sqlite:///data/cache.db
        └── table (value): cache

